# Procedural Floorplan Generator

In [ ]:
import matplotlib
matplotlib.use('Agg')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyBboxPatch
import random
from dataclasses import dataclass
from IPython.display import display

print("Imports done!")

In [ ]:
ROOM_COLORS = {
    "office": "#E3F2FD",
    "conference": "#FFF3E0",
    "lobby": "#E8F5E9",
    "restroom": "#FCE4EC",
    "kitchen": "#FFF8E1",
    "server": "#E0E0E0",
    "storage": "#EFEBE9",
    "elevator": "#B0BEC5",
    "stairs": "#CFD8DC",
    "open_office": "#E1F5FE",
    "lab": "#E0F7FA",
}

@dataclass
class Room:
    x: int
    y: int
    width: int
    height: int
    room_type: str
    name: str = ""

print("Room types defined!")

In [ ]:
def generate_office_floor(width=60, height=40, seed=None):
    if seed: random.seed(seed)
    rooms = []
    corridor_y = height // 2 - 2
    corridor_h = 4
    
    rooms.append(Room(2, corridor_y - 6, 10, 16, "lobby", "Lobby"))
    cx = width // 2 - 4
    rooms.append(Room(cx, corridor_y - 5, 4, 5, "elevator", "Elev"))
    rooms.append(Room(cx + 4, corridor_y - 5, 4, 5, "stairs", "Stairs"))
    rooms.append(Room(cx, corridor_y + corridor_h, 4, 5, "restroom", "WC"))
    rooms.append(Room(cx + 4, corridor_y + corridor_h, 4, 5, "restroom", "WC"))
    
    x, num = 14, 101
    while x < width - 8:
        if cx - 2 <= x <= cx + 10: x = cx + 11; continue
        w = random.choice([6, 7, 8])
        if x + w > width - 2: break
        rooms.append(Room(x, 2, w, corridor_y - 3, random.choice(["office"]*3 + ["conference"]), str(num)))
        x += w + 1; num += 1
    
    x, num = 14, 201
    while x < width - 8:
        if cx - 2 <= x <= cx + 10: x = cx + 11; continue
        w = random.choice([6, 7, 8])
        if x + w > width - 2: break
        rooms.append(Room(x, corridor_y + corridor_h + 1, w, height - corridor_y - corridor_h - 3, 
                         random.choice(["office"]*2 + ["conference", "kitchen"]), str(num)))
        x += w + 1; num += 1
    
    return rooms, width, height, [(2, corridor_y, width - 4, corridor_h)]

def generate_open_plan(width=60, height=40, seed=None):
    if seed: random.seed(seed)
    rooms = []
    margin = 8
    rooms.append(Room(margin, margin, width - 2*margin, height - 2*margin, "open_office", "Open Office"))
    for i in range(2, width - 6, 7):
        rooms.append(Room(i, 1, 6, margin - 2, random.choice(["office", "conference"])))
        rooms.append(Room(i, height - margin + 1, 6, margin - 2, random.choice(["office", "conference"])))
    cx, cy = width // 2 - 6, height // 2 - 4
    rooms.append(Room(cx, cy, 4, 4, "elevator"))
    rooms.append(Room(cx + 4, cy, 4, 4, "stairs"))
    rooms.append(Room(cx + 8, cy, 4, 4, "restroom"))
    rooms.append(Room(cx, cy + 4, 6, 4, "kitchen"))
    rooms.append(Room(cx + 6, cy + 4, 6, 4, "server"))
    return rooms, width, height, []

def generate_lab_floor(width=70, height=50, seed=None):
    if seed: random.seed(seed)
    rooms = []
    corridors = [(2, height // 2 - 2, width - 4, 4), (width // 3, 2, 4, height - 4)]
    for i, y in enumerate(range(2, height // 2 - 4, 10)):
        rooms.append(Room(2, y, width // 3 - 4, 9, "lab", f"Lab {i+1}"))
    for i, y in enumerate(range(height // 2 + 4, height - 10, 10)):
        rooms.append(Room(2, y, width // 3 - 4, 9, "lab", f"Lab {i+3}"))
    x = width // 3 + 6
    while x < width - 10:
        w = random.choice([7, 8, 9])
        rooms.append(Room(x, 2, w, height // 2 - 5, random.choice(["office", "conference"])))
        rooms.append(Room(x, height // 2 + 3, w, height // 2 - 5, random.choice(["office", "storage"])))
        x += w + 1
    rooms.append(Room(width - 10, height // 2 - 6, 8, 5, "elevator"))
    rooms.append(Room(width - 10, height // 2 + 3, 8, 5, "stairs"))
    return rooms, width, height, corridors

def generate_random_floor(width=60, height=40, seed=None):
    if seed: random.seed(seed)
    rooms = []
    def split(x, y, w, h, depth=0):
        if depth > 3 or w < 10 or h < 10:
            rooms.append(Room(x + 1, y + 1, w - 2, h - 2, random.choice(list(ROOM_COLORS.keys()))))
            return
        if random.random() < 0.25 and depth > 1:
            rooms.append(Room(x + 1, y + 1, w - 2, h - 2, random.choice(list(ROOM_COLORS.keys()))))
            return
        if w > h:
            s = random.randint(w // 3, 2 * w // 3)
            split(x, y, s, h, depth + 1); split(x + s, y, w - s, h, depth + 1)
        else:
            s = random.randint(h // 3, 2 * h // 3)
            split(x, y, w, s, depth + 1); split(x, y + s, w, h - s, depth + 1)
    split(2, 2, width - 4, height - 4)
    rooms.append(Room(width // 2 - 3, height // 2 - 3, 3, 3, "elevator"))
    rooms.append(Room(width // 2, height // 2 - 3, 3, 3, "stairs"))
    return rooms, width, height, []

print("Generators ready!")

In [ ]:
STYLES = {
    "colorful": {"bg": "#eceff1", "wall": "#37474f", "corridor": "#b0bec5"},
    "blueprint": {"bg": "#1a237e", "wall": "#e3f2fd", "corridor": "#3949ab"},
    "modern": {"bg": "#263238", "wall": "#80cbc4", "corridor": "#455a64"},
}

def render_floor(rooms, width, height, corridors, title="Floor", style="colorful"):
    s = STYLES.get(style, STYLES["colorful"])
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_facecolor(s["bg"])
    
    for cx, cy, cw, ch in corridors:
        ax.add_patch(Rectangle((cx, cy), cw, ch, facecolor=s["corridor"], edgecolor=s["wall"], lw=0.5))
    
    for room in rooms:
        color = ROOM_COLORS.get(room.room_type, "#fff") if style == "colorful" else s["bg"]
        ax.add_patch(FancyBboxPatch((room.x, room.y), room.width, room.height,
            boxstyle="round,pad=0.02", facecolor=color, edgecolor=s["wall"], lw=1.5))
        if room.width > 4 and room.height > 3:
            ax.text(room.x + room.width/2, room.y + room.height/2, 
                   room.name or room.room_type[:6], ha='center', va='center', 
                   fontsize=7, color="#333" if style == "colorful" else s["wall"], weight='bold')
    
    ax.add_patch(Rectangle((0, 0), width, height, fill=False, edgecolor=s["wall"], lw=3))
    ax.set_xlim(-2, width + 2); ax.set_ylim(-2, height + 2)
    ax.set_aspect('equal'); ax.axis('off')
    ax.set_title(title, fontsize=14, fontweight='bold', color=s["wall"], pad=10)
    plt.tight_layout()
    return fig

def render_multifloor(floors_data, title="Building", style="colorful"):
    n = len(floors_data)
    cols = min(n, 2)
    rows = (n + cols - 1) // cols
    s = STYLES.get(style, STYLES["colorful"])
    
    fig, axes = plt.subplots(rows, cols, figsize=(12 * cols, 8 * rows))
    fig.patch.set_facecolor(s["bg"])
    axes = [axes] if n == 1 else axes.flatten()
    
    for i, (rooms, width, height, corridors, floor_num) in enumerate(floors_data):
        ax = axes[i]
        ax.set_facecolor(s["bg"])
        for cx, cy, cw, ch in corridors:
            ax.add_patch(Rectangle((cx, cy), cw, ch, facecolor=s["corridor"], edgecolor=s["wall"], lw=0.5))
        for room in rooms:
            color = ROOM_COLORS.get(room.room_type, "#fff") if style == "colorful" else s["bg"]
            ax.add_patch(FancyBboxPatch((room.x, room.y), room.width, room.height,
                boxstyle="round,pad=0.02", facecolor=color, edgecolor=s["wall"], lw=1.5))
            if room.width > 4 and room.height > 3:
                ax.text(room.x + room.width/2, room.y + room.height/2,
                       room.name or room.room_type[:6], ha='center', va='center',
                       fontsize=7, color="#333" if style == "colorful" else s["wall"], weight='bold')
        ax.add_patch(Rectangle((0, 0), width, height, fill=False, edgecolor=s["wall"], lw=3))
        ax.set_xlim(-2, width + 2); ax.set_ylim(-2, height + 2)
        ax.set_aspect('equal'); ax.axis('off')
        ax.set_title(f"Floor {floor_num}", fontsize=12, fontweight='bold', color=s["wall"])
    
    for i in range(n, len(axes)): axes[i].axis('off')
    fig.suptitle(title, fontsize=16, fontweight='bold', color=s["wall"], y=0.98)
    plt.tight_layout()
    return fig

print("Renderers ready!")

---
## Generate Single Floors

In [ ]:
rooms, w, h, corr = generate_office_floor(seed=42)
fig = render_floor(rooms, w, h, corr, "Office Building", "colorful")
display(fig)
plt.close(fig)

In [ ]:
rooms, w, h, corr = generate_open_plan(seed=42)
fig = render_floor(rooms, w, h, corr, "Open Plan Office", "colorful")
display(fig)
plt.close(fig)

In [ ]:
rooms, w, h, corr = generate_lab_floor(seed=42)
fig = render_floor(rooms, w, h, corr, "Research Lab", "blueprint")
display(fig)
plt.close(fig)

In [ ]:
rooms, w, h, corr = generate_random_floor(seed=42)
fig = render_floor(rooms, w, h, corr, "Random Layout", "modern")
display(fig)
plt.close(fig)

---
## Multi-Floor Building

In [ ]:
generators = [generate_office_floor, generate_open_plan, generate_lab_floor, generate_random_floor]
floors_data = [(lambda g=g, i=i: (*g(seed=100+i), i+1))() for i, g in enumerate(generators)]

fig = render_multifloor(floors_data, "4-Floor Office Complex", "colorful")
display(fig)
plt.close(fig)

---
## Batch Random Buildings

In [ ]:
generators = [generate_office_floor, generate_open_plan, generate_lab_floor, generate_random_floor]
styles = ["colorful", "blueprint", "modern"]

for i in range(6):
    seed = np.random.randint(0, 10000)
    rooms, w, h, corr = random.choice(generators)(seed=seed)
    fig = render_floor(rooms, w, h, corr, f"Building #{i+1} (seed={seed})", random.choice(styles))
    display(fig)
    plt.close(fig)
    print(f"Done {i+1}/6")

---
## Export to Numpy (for sbsim)

In [ ]:
def to_numpy(rooms, width, height, scale=4):
    """0=interior, 1=wall, 2=exterior"""
    arr = np.full((height * scale, width * scale), 2, dtype=np.uint8)
    arr[scale:-scale, scale:-scale] = 0
    for room in rooms:
        x1, y1 = room.x * scale, room.y * scale
        x2, y2 = (room.x + room.width) * scale, (room.y + room.height) * scale
        arr[y1:y1+1, x1:x2] = 1
        arr[y2-1:y2, x1:x2] = 1
        arr[y1:y2, x1:x1+1] = 1
        arr[y1:y2, x2-1:x2] = 1
        cx, cy = (room.x + room.width // 2) * scale, (room.y + room.height) * scale - 1
        arr[max(0,cy-1):cy+2, max(0,cx-1):cx+2] = 0
    return arr

rooms, w, h, _ = generate_office_floor(seed=123)
floor_array = to_numpy(rooms, w, h)
print(f"Shape: {floor_array.shape}, Values: 0=interior, 1=wall, 2=exterior")

fig, ax = plt.subplots(figsize=(12, 8))
ax.imshow(floor_array, cmap='Blues', interpolation='nearest')
ax.set_title("Simulation Array"); ax.axis('off')
display(fig)
plt.close(fig)

np.save("generated_floorplan.npy", floor_array)
print("Saved to generated_floorplan.npy")